In [ ]:
#dependencies
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import plotly.express as px
import matplotlib.pyplot as pl
import cv2
import os
import random
import timm
from tqdm.auto import tqdm
import albumentations as alb
from albumentations.pytorch import ToTensorV2
from timm.data.auto_augment import rand_augment_transform

In [ ]:
class ModelNet(nn.Module):
    def __init__(self, arc, pretrained=True):
        super().__init__()
        self.model_name = arc.model_name
        self.model = timm.create_model(arc.model_name, pretrained=pretrained, num_classes=8)
    
    def forward(self, x):
        x=self.model(x)
        return x
    


class MyModel():
    def __init__(self, arc, modelnet):
        super().__init__()
        self.model = modelnet(arc, pretrained=True)
        self.arc=arc
        
        if arc.optimizer=='adam':
            self.optimizer=torch.optim.Adam(self.model.parameters(), lr=arc.lr, weight_decay=0.001)
        if arc.optimizer=='rmsprop':
            self.optimizer=torch.optim.RMSprop(self.model.parameters(), lr=arc.lr, weight_decay=0.001, momentum=0.9)
        if arc.optimizer == 'sgd':
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=arc.lr,weight_decay=0.001, momentum=0.9)
        if arc.criterion=='crossentropyloss':
            self.criterion = nn.CrossEntropyLoss()
        if arc.criterion=='bce':
            self.criterion = nn.BCEWithLogitsLoss()
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = self.optimizer, gamma=0.9)

    def trainer(self, train_loader, model, optimizer, criterion):
        model.to('cuda').train()
        it_loss=0
        counter=0
        for data in train_loader:
            images = data['image'].to('cuda')
            labels = data['target'].to('cuda')

            output = model(images)
            #print(output.shape)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            

            it_loss+=loss.item()*images.shape[0]
            counter+=images.shape[0]

        return it_loss/counter

    def tester(self, test_loader, model, criterion):
        model.to('cuda').eval()
        loss=0
        counter=0
        for data in test_loader:
            images=data['image'].to('cuda')
            labels=data['target'].to('cuda')

            with torch.no_grad():
                out = model(images)
                single_loss = criterion(out, labels)
                loss+=single_loss*images.shape[0]
                counter+=images.shape[0]
            #self.scheduler.step(loss/counter)

        return loss/counter

    def SHAKTI(self):
        best_loss = np.inf
        l=[]
        for it in range(self.arc.iterations):
            print(f'Began iteration {it+1}')
            train_loss = self.trainer(self.arc.train_loader, self.model, self.optimizer, self.criterion)
            test_loss = self.tester(self.arc.test_loader, self.model, self.criterion)
            l.append(test_loss.to('cpu'))
            self.scheduler.step()
            if test_loss < best_loss:
                print('Improved')
                best_loss = test_loss
                print(f'Iteration {it+1} - Test_Loss: {best_loss}')
                KARTIKAY = {'model':self.model.state_dict(),
                            'optimizer':self.optimizer.state_dict()}
                torch.save(KARTIKAY, './'+f'{self.arc.model_name}_best.pth')
                print("Updated model saved")
            else:
                print('Not Improved')
                print(f'Iteration {it+1} - Test_Loss: {test_loss}')
            print(f'Ended iteration {it+1}')
        pl.plot(list(range(len(l))), l)
        pl.show()
        
        return KARTIKAY


In [ ]:
#dataset class
class CustomDataset:
    def __init__(self, image_paths, targets, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        target = self.targets[:, idx]        #normal case-0 bengin case-1 malignant case-2
        image = cv2.imread(self.image_paths[idx])  #bgr format h
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #bgr -> rgb
        if self.augmentations != None:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        
        return {
            'image':torch.tensor(image),
            'target':torch.tensor(target)
        }
    
class TestData():
    def __init__(self, image_paths, augmentations=None):
        self.image_paths = image_paths
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])  #bgr format h
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #bgr -> rgb
        if self.augmentations != None:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        return {
            'image':torch.tensor(image)
        }


In [ ]:
def LoadDatasets(batch_size, augmentation, CustomDataset=CustomDataset):
    # read csv
    df_train = pd.read_csv("/kaggle/input/ai-of-god-v20/train.csv",index_col=False)
    
    filenames = list(df_train['FileName'])
    
    labels = list(df_train['Class'])
    m = len(labels)
    image_paths=[]
    num_classes=8
    path='/kaggle/input/ai-of-god-v20/train/'
    for file in filenames:
        img_path = path+file
        image_paths.append(img_path)
    
    random.seed(0)
    random.shuffle(image_paths)
    random.seed(0)
    random.shuffle(labels)
    
    classes = [(0, 'Other'), (1, 'Crater'), (2, 'Dark Dune'), (3, 'Slope Streak'), (4, 'Bright Dune'), 
               (5, 'Impact Ejecta'), (6, 'Swiss Cheese'), (7, 'Spider')]
    
    labels_enc=np.zeros((num_classes, m))
    for i in range(m):
        labels_enc[labels[i], i]=1
        
    train_data, train_labels, test_data, test_labels = image_paths[0:7701], labels_enc[:, 0:7701], image_paths[7701:8201], labels_enc[:, 7701:8201]
   # augmentation = timm.data.transforms_factory.create_transform(224, is_training=True, auto_augment='rand-m9-mstd0.5')
    train_dataset = CustomDataset(train_data, train_labels, augmentation)
    test_dataset = CustomDataset(test_data, test_labels, augmentation)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2)
    
    return train_dataset, test_dataset, train_loader, test_loader

In [ ]:
augmentation = alb.Compose(
    [
        alb.Resize(256,256),
        alb.CenterCrop(224,224),
        #alb.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.5),
        alb.Rotate(limit=45, p=0.6),
        alb.HorizontalFlip(p=0.6),
        alb.RandomBrightnessContrast(p=0.6),
        alb.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]
)
train_dataset, test_dataset, train_loader, test_loader=LoadDatasets(64, augmentation)

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
train_dataset[0]

In [ ]:
class ModelArchitecture():
    def __init__(self, inspiration, train_loader, test_loader):
        if inspiration == 'AGNI':
            self.model_name='vgg19'
            self.batch_size=16
            self.iterations=10
            self.lr=0.00001
            self.optimizer='adam'
            self.trainagain=False
            self.criterion='crossentropyloss'
            self.train_loader = train_loader
            self.test_loader=test_loader
        if inspiration == 'VAYU':
            self.model_name='resnet50'
            self.batch_size=32
            self.iterations=10
            self.lr=0.0001
            self.optimizer='adam'
            self.trainagain=False
            self.criterion='crossentropyloss'
            self.train_loader = train_loader
            self.test_loader=test_loader
        if inspiration == 'PAWAN':
            self.model_name='inception_v3'
            self.batch_size=16
            self.iterations=10
            self.lr=0.00001
            self.optimizer='rmsprop'
            self.trainagain=False
            self.criterion='bce'
            self.train_loader = train_loader
            self.test_loader=test_loader
        if inspiration == 'VARUN':
            self.model_name='densenet201'
            self.batch_size=16
            self.iterations=10
            self.lr=0.0001
            self.optimizer='adam'
            self.trainagain=False
            self.criterion='crossentropyloss'
            self.train_loader = train_loader
            self.test_loader=test_loader
        if inspiration == 'INDR':
            self.model_name='repvgg_b1g4'
            self.batch_size=16
            self.iterations=10
            self.lr=0.0001
            self.optimizer='rmsprop'
            self.trainagain=False
            self.criterion='crossentropyloss'
            self.train_loader = train_loader
            self.test_loader=test_loader
        if inspiration == 'BRAHMA':
            self.model_name='inception_resnet_v2'
            self.batch_size=16
            self.iterations=10
            self.lr=0.0001
            self.optimizer='rmsprop'
            self.trainagain=False
            self.criterion='crossentropyloss'
            self.train_loader = train_loader
            self.test_loader=test_loader

In [ ]:
VISHNU = ModelArchitecture('VAYU', train_loader, test_loader)

In [ ]:
if VISHNU.trainagain:
    check_point = torch.load('./'+f'{VISHNU.model_name}_best.pth')
    pastSHIV = ModelNet(VISHNU, pretrained=True)
    pastSHIV.load_state_dict(check_point['model'])
    SHIV = MyModel(VISHNU, ModelNet)
    SHIV.model = pastSHIV
    SHIV.optimizer.load_state_dict(check_point['optimizer'])
   #SHIV.scheduler.load_state_dict(check_point['scheduler']) 
    print("Loaded past SHIV")
else:
    SHIV = MyModel(VISHNU, ModelNet)
    print('Loaded SHIV')

In [ ]:
KARTIKAY = SHIV.SHAKTI()

In [ ]:
df_test = pd.read_csv("/kaggle/input/ai-of-god-v20/test.csv")
filenames = list(df_test['FileName'])
image_paths = []
num_classes=8
path='/kaggle/input/ai-of-god-v20/test/'
for file in filenames:
    img_path = path+file    
    image_paths.append(img_path)

some_dataset = TestData(image_paths, augmentation)

TADKASUR = torch.utils.data.DataLoader(some_dataset, batch_size=64)

In [ ]:
def YUDH(TADKASUR, SKAND):
    SKAND.to('cuda').eval()
    
    SWARG=[]
    for asur in TADKASUR:
        heart = asur['image'].to('cuda')
        with torch.no_grad():
            out = SKAND(heart)
            SWARG.append(out.softmax(1).to('cpu').numpy())
            
    SWARG = np.concatenate(SWARG)
    SWARG = SWARG.argmax(1)
    return SWARG

In [ ]:
check_point = torch.load('./'+f'{VISHNU.model_name}_best.pth')
SKAND = ModelNet(VISHNU, pretrained=True)
SKAND.to('cuda')
SKAND.load_state_dict(check_point['model'])

In [ ]:
SWARG = YUDH(TADKASUR, SKAND)

In [ ]:
len(SWARG)

In [ ]:
sub = pd.read_csv("/kaggle/input/ai-of-god-v20/test.csv")
sub['Class'] = SWARG

In [ ]:
sub

In [ ]:
sub.to_csv('./submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as pl
?pl.plot